In [ ]:
!git clone https://github.com/notRobot200/CycleMe.git

Cloning into 'CycleMe'...
remote: Enumerating objects: 6594, done.
remote: Counting objects: 100% (853/853), done.
remote: Compressing objects: 100% (840/840), done.
remote: Total 6594 (delta 18), reused 844 (delta 13), pack-reused 5741
Receiving objects: 100% (6594/6594), 229.59 MiB | 25.65 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Updating files: 100% (4283/4283), done.


In [ ]:
import tensorflow as tf

DIR = "/content/CycleMe/CycleMeDataset"
dataset = tf.keras.preprocessing.image_dataset_from_directory(DIR)

Found 4280 files belonging to 8 classes.


In [ ]:
from pathlib import Path
import imghdr

data_dir = "/content/CycleMe/CycleMeDataset"
image_extensions = [".png", ".jpg", ".jpeg", ".GIF"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

/content/CycleMe/CycleMeDataset/Aluminium/alum (290).jpg is not an image
/content/CycleMe/CycleMeDataset/Aluminium/alum (271).jpg is not an image
/content/CycleMe/CycleMeDataset/Textiles/text (29).jpg is not an image
/content/CycleMe/CycleMeDataset/Textiles/text (295).jpg is not an image
/content/CycleMe/CycleMeDataset/Textiles/text (19).jpg is not an image
/content/CycleMe/CycleMeDataset/Textiles/text (199).jpg is not an image
/content/CycleMe/CycleMeDataset/Plastic/plast (12).jpg is not an image
/content/CycleMe/CycleMeDataset/Glass/glass (10).jpg is not an image
/content/CycleMe/CycleMeDataset/Glass/glass (14).jpg is not an image
/content/CycleMe/CycleMeDataset/Glass/glass (11).jpg is not an image
/content/CycleMe/CycleMeDataset/Glass/glass (7).jpg is not an image
/content/CycleMe/CycleMeDataset/Glass/glass (336).jpg is not an image
/content/CycleMe/CycleMeDataset/Glass/glass (356).jpg is not an image
/content/CycleMe/CycleMeDataset/Glass/glass (13).jpg is not an image
/content/Cycl

In [ ]:
from pathlib import Path
import imghdr
import os

from pathlib import Path
import imghdr

img_link=list(Path("/content/CycleMe/CycleMeDataset").glob(r'**/*.jpg'))

count_num=0
for lnk in img_link:
    binary_img=open(lnk,'rb')
    find_img=tf.compat.as_bytes('JFIF') in binary_img.peek(10)#The JFIF is a JPEG File Interchange Format (JFIF). It is a standard which we gauge if an image is corrupt or substandard
    if not find_img:
        count_num+=1
        os.remove(str(lnk))
print('Total %d pcs image delete from Dataset' % count_num)
#this should help you delete the bad encoded

Total 79 pcs image delete from Dataset


In [ ]:
import os

labels = ['Aluminium', 'Carton', 'Glass', 'Organic Waste', 'Paper and Cardboard', 'Plastic', 'Styrofoam', 'Textiles']
for label in labels:
    directory = os.path.join(DIR, label)
    print("Images of label \"" + label + "\":\t", len(os.listdir(directory)))

Images of label "Aluminium":	 598
Images of label "Carton":	 323
Images of label "Glass":	 585
Images of label "Organic Waste":	 542
Images of label "Paper and Cardboard":	 601
Images of label "Plastic":	 593
Images of label "Styrofoam":	 385
Images of label "Textiles":	 574


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(DIR, 
                                                                    validation_split=0.1, 
                                                                    subset="training", 
                                                                    seed=42, 
                                                                    batch_size=128, 
                                                                    smart_resize=True, 
                                                                    image_size=(256, 256))
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(DIR, 
                                                                   validation_split=0.1, 
                                                                   subset="validation", 
                                                                   seed=42, 
                                                                   batch_size=128, 
                                                                   smart_resize=True, 
                                                                   image_size=(256, 256))

classes = train_dataset.class_names
numClasses = len(train_dataset.class_names)
print(classes)

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

Found 4201 files belonging to 8 classes.
Using 3781 files for training.
Found 4201 files belonging to 8 classes.
Using 420 files for validation.
['Aluminium', 'Carton', 'Glass', 'Organic Waste', 'Paper and Cardboard', 'Plastic', 'Styrofoam', 'Textiles']


In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.inception_v3 import InceptionV3

pretrain_model = InceptionV3(
                 include_top=False, 
                 weights='imagenet', 
                 input_shape=(256, 256, 3))

pretrain_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 127, 127, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_95 (BatchN  (None, 127, 127, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                      

In [ ]:
local_weights_file = '/content/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pretrain_model.load_weights(local_weights_file)
# Freeze the weights of the layers.
for layer in pretrain_model.layers:
  layer.trainable = False

In [ ]:
# grader-required-cell

# GRADED FUNCTION: output_of_last_layer
def output_of_last_layer(pretrain_model):
  """
  Gets the last layer output of a model
  
  Args:
    pre_trained_model (tf.keras Model): model to get the last layer output from
    
  Returns:
    last_output: output of the model's last layer 
  """
  ### START CODE HERE
  last_desired_layer = pretrain_model.layers[-1]
  print('last layer output shape: ', last_desired_layer.output_shape)
  last_output = last_desired_layer.output
  print('last layer output: ', last_output)
  ### END CODE HERE

  return last_output

In [ ]:
# grader-required-cell

last_output = output_of_last_layer(pretrain_model)

last layer output shape:  (None, 6, 6, 2048)
last layer output:  KerasTensor(type_spec=TensorSpec(shape=(None, 6, 6, 2048), dtype=tf.float32, name=None), name='mixed10/concat:0', description="created by layer 'mixed10'")


In [ ]:
# grader-required-cell
from tensorflow.keras import layers
# GRADED FUNCTION: create_final_model
def create_final_model(pretrain_model, last_output):
  """
  Appends a custom model to a pre-trained model
  
  Args:
    pre_trained_model (tf.keras Model): model that will accept the train/test inputs
    last_output (tensor): last layer output of the pre-trained model
    
  Returns:
    model: the combined model
  """
  # Flatten the output layer to 1 dimension
  #x = layers.Flatten()(last_output)
  x = layers.Dropout(0.2)(last_output) # jk dropout besar, akurasi menurun
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.BatchNormalization()(x)

  ### START CODE HERE

  # Add a fully connected layer with 1024 hidden units and ReLU activation
  x = layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.03))(x)
                   #activity_regularizer=tf.keras.regularizers.l1(0.045),
                   #kernel_initializer='he_uniform')(x)

  # Add a dropout rate of 0.3
  x = layers.Dropout(0.2)(x)  
  # Add a final sigmoid layer for classification
  x = layers.Dense(numClasses, activation='softmax')(x)        

  # Create the complete model by using the Model class
  model = tf.keras.Model(inputs=pretrain_model.input, outputs=x)

  # Compile the model
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), 
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics = ['accuracy'])

  ### END CODE HERE
  
  return model

In [ ]:
# from keras.optimizers import Adam 
model = create_final_model(pretrain_model, last_output)

# model.compile(loss='sparse_categorical_crossentropy', 
#               optimizer=Adam(1e-04),
#               metrics=['acc'])

In [ ]:
# Train the model.
history = model.fit(
            train_dataset,
            validation_data = test_dataset,
            epochs = 20,
            verbose = 2)

Epoch 1/20
30/30 - 726s - loss: 32.7835 - accuracy: 0.2975 - val_loss: 10.9518 - val_accuracy: 0.1738 - 726s/epoch - 24s/step
Epoch 2/20
30/30 - 740s - loss: 6.6375 - accuracy: 0.2624 - val_loss: 4.7192 - val_accuracy: 0.1833 - 740s/epoch - 25s/step
Epoch 3/20
30/30 - 727s - loss: 4.4087 - accuracy: 0.2433 - val_loss: 4.3190 - val_accuracy: 0.2405 - 727s/epoch - 24s/step
Epoch 4/20
30/30 - 716s - loss: 4.2681 - accuracy: 0.2417 - val_loss: 4.3101 - val_accuracy: 0.2476 - 716s/epoch - 24s/step
Epoch 5/20
30/30 - 718s - loss: 4.2701 - accuracy: 0.2404 - val_loss: 4.2925 - val_accuracy: 0.2190 - 718s/epoch - 24s/step
Epoch 6/20


KeyboardInterrupt: ignored